# Camada Delivery
Adaptação dos dados para consumo

## dlzd_olist_order_reviews
Transformações baseadas na camada Trusted para:
* Obter tokens a partir dos textos de review_comment_title e review_comment_message
* Gerar nova coluna review_comment_title_and_message com os tokens do título e comentário de uma mesma avaliação
* Padronizar tokens:
  * Lower case
  * Remover palavras comuns e conectores
  * Desconsiderar de palavras com 1 caractere e mais de 15 caracteres
  * Desconsiderar de espaços em branco, números e caracteres especiais


Importando bibliotecas necessárias

In [16]:
from gensim.utils import simple_preprocess
import pandas as pd

In [17]:
pd.set_option('display.max_colwidth', None)

Lendo dados da camada trusted

In [18]:
dlzd_olist_order_reviews = pd.read_parquet(
    path='../dataset/trusted/trzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Utilizando o pacote *simple_preprocess*, responsável por gerar uma lista de tokens

Tranformações aplicadas:
*   Padronização em lower case
*   Desconsideração de palavras com 1 caracter e mais de 15 caracteres
*   Desconsideração de espaços em branco, números e caracteres especiais


Doc: https://tedboy.github.io/nlps/generated/generated/gensim.utils.simple_preprocess.html

In [19]:
dlzd_olist_order_reviews['review_comment_title'] = dlzd_olist_order_reviews['review_comment_title'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

dlzd_olist_order_reviews['review_comment_message'] = dlzd_olist_order_reviews['review_comment_message'].apply(
    lambda sentence: simple_preprocess(sentence, deacc=True) if sentence else []
)

Unindo título com comentários

In [20]:
titles = dlzd_olist_order_reviews['review_comment_title']
messages = dlzd_olist_order_reviews['review_comment_message']

dlzd_olist_order_reviews['review_comment_title_and_message'] = titles + messages

Removendo linhas sem palavras

In [31]:
empty_reviews_mask =  dlzd_olist_order_reviews['review_comment_title_and_message'].apply(lambda tokens: len(tokens)) == 0

In [33]:
empty_reviews_index = dlzd_olist_order_reviews.loc[empty_reviews_mask].index

In [35]:
dlzd_olist_order_reviews.drop(
    index=empty_reviews_index,
    inplace=True
)

Criando a coluna review_sentiment a partir da coluna review_score, onde:
*   -1, para notas menor que 3
*   0, para notas igual à 3
*   +1, para notas mais que 3



In [37]:
dlzd_olist_order_reviews['review_sentiment'] = (
    dlzd_olist_order_reviews['review_score']
    .apply(lambda x: -1 if x < 3 else (0 if x == 3 else 1))
)

Reorganizando ordem das colunas

In [39]:
dlzd_olist_order_reviews = dlzd_olist_order_reviews.loc[:,
    [
        'review_id',
        'order_id',
        'review_score',
        'review_sentiment',
        'review_comment_title',
        'review_comment_message',
        'review_comment_title_and_message',
        'review_creation_date',
        'review_answer_timestamp'
    ]
]

Visualizando dataset transformado

In [40]:
dlzd_olist_order_reviews.head()

,review_id,order_id,review_score,review_sentiment,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
0,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,1,[],"[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]","[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]",2018-03-01,2018-03-02 10:26:53
1,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,1,[recomendo],"[aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]","[recomendo, aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]",2018-05-22,2018-05-23 16:45:47
2,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,1,[],"[mas, um, pouco, travando, pelo, valor, ta, boa]","[mas, um, pouco, travando, pelo, valor, ta, boa]",2018-02-16,2018-02-20 10:52:22
3,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,1,[otimo],"[loja, nota]","[otimo, loja, nota]",2018-07-10,2018-07-11 14:10:25
4,0e0190b9db53b689b285d3f3916f8441,79832b7cb59ac6f887088ffd686e1d5e,5,1,[],"[obrigado, pela, atencao, amim, dispensada]","[obrigado, pela, atencao, amim, dispensada]",2017-12-01,2017-12-09 22:58:58


Salvando dados na camada delivery em formato parquet

In [41]:
dlzd_olist_order_reviews.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

## dlzd_olist_order_reviews_clean
Transformações baseadas na tabela dlzd_olist_order_reviews para remoção de stop words presentes nos tokens das colunas:
* review_comment_title
* review_comment_message
* review_comment_title_and_message


Importação das bibliotecas

In [ ]:
!pip install spacy
!python -m spacy download pt_core_news_sm

In [21]:
from gensim.utils import simple_preprocess
import pandas as pd
import spacy

Leitura da tabela dlzd_olist_order_reviews

In [22]:
dlzd_olist_order_reviews_clean = pd.read_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy',
    engine='pyarrow',
)

Remoção de stop words

In [23]:
def remove_stopwords(tokens):
    return [token for token in tokens if token not in stop_words]

In [24]:
# conjunto de stop words a partir do modelo SpaCy
nlp = spacy.load('pt_core_news_sm')
stop_words = set(nlp.Defaults.stop_words)

In [25]:
dlzd_olist_order_reviews_clean['review_comment_title_and_message'] = dlzd_olist_order_reviews_clean['review_comment_title_and_message'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean['review_comment_message'] = dlzd_olist_order_reviews_clean['review_comment_message'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean['review_comment_title'] = dlzd_olist_order_reviews_clean['review_comment_title'].apply(remove_stopwords)
dlzd_olist_order_reviews_clean

,review_id,order_id,review_score,review_sentiment,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
0,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,1,[],"[parabens, lojas, lannister, adorei, comprar, internet, seguro, pratico, parabens, feliz, pascoa]","[parabens, lojas, lannister, adorei, comprar, internet, seguro, pratico, parabens, feliz, pascoa]",2018-03-01,2018-03-02 10:26:53
1,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,1,[recomendo],"[aparelho, eficiente, site, marca, aparelho, impresso, desinfector, chegar, outro, nome, atualizar, marca, correta, aparelho]","[recomendo, aparelho, eficiente, site, marca, aparelho, impresso, desinfector, chegar, outro, nome, atualizar, marca, correta, aparelho]",2018-05-22,2018-05-23 16:45:47
2,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,1,[],"[travando, ta]","[travando, ta]",2018-02-16,2018-02-20 10:52:22
3,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,1,[otimo],"[loja, nota]","[otimo, loja, nota]",2018-07-10,2018-07-11 14:10:25
4,0e0190b9db53b689b285d3f3916f8441,79832b7cb59ac6f887088ffd686e1d5e,5,1,[],"[atencao, amim, dispensada]","[atencao, amim, dispensada]",2017-12-01,2017-12-09 22:58:58
...,...,...,...,...,...,...,...,...,...
20924,0e7bc73fde6782891898ea71443f9904,bd78f91afbb1ecbc6124974c5e813043,4,1,[],[aprovado],[aprovado],2018-07-04,2018-07-05 00:25:13
20925,58be140ccdc12e8908ff7fd2ba5c7cb0,0ebf8e35b9807ee2d717922d5663ccdb,5,1,[produto],"[ficamos, satisfeitos, produto, atende, necessidades, muita, facilidade, recomendo]","[produto, ficamos, satisfeitos, produto, atende, necessidades, muita, facilidade, recomendo]",2018-06-30,2018-07-02 23:09:35
20926,40743b46a0ee86375cedb95e82b78d75,3e93213bb8fdda91186b4018b2fe0030,5,1,"[otima, embalagem]",[],"[otima, embalagem]",2018-08-08,2018-08-08 16:56:16
20927,2ee221b28e5b6fceffac59487ed39348,f2d12dd37eaef72ed7b1186b2edefbcd,2,-1,"[foto, enganosa]","[foto, diferente, principalmente, graninha, sintetica, rala, diferente, foto]","[foto, enganosa, foto, diferente, principalmente, graninha, sintetica, rala, diferente, foto]",2018-03-28,2018-05-25 01:23:26


In [26]:
dlzd_olist_order_reviews_clean.count()

,0
review_id,20929
order_id,20929
review_score,20929
review_sentiment,20929
review_comment_title,20929
review_comment_message,20929
review_comment_title_and_message,20929
review_creation_date,20929
review_answer_timestamp,20929


Salvamento de arquivo parquet na camada delivery

In [27]:
dlzd_olist_order_reviews_clean.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews_clean.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=False,
)

# dlzd_olist_order_reviews_training e dlzd_olist_order_reviews_test
Transformações baseadas na tabela dlzd_olist_order_reviews para criação dos conjuntos de treinamento e teste utilizados pelos modelos.

Importações de bibliotecas

In [42]:
from sklearn.model_selection import train_test_split
import pandas as pd

Leitura do arquivo dlzd_olist_order_reviews

In [43]:
dlzd_olist_order_reviews = pd.read_parquet('../dataset/delivery/dlzd_olist_order_reviews.parquet.snappy')

In [44]:
dlzd_olist_order_reviews.set_index('review_id', inplace=True)

In [47]:
dlzd_olist_order_reviews.head()

,order_id,review_score,review_sentiment,review_comment_title,review_comment_message,review_comment_title_and_message,review_creation_date,review_answer_timestamp
review_id,,,,,,,,
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,1,[],"[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]","[parabens, lojas, lannister, adorei, comprar, pela, internet, seguro, pratico, parabens, todos, feliz, pascoa]",2018-03-01,2018-03-02 10:26:53
8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,1,[recomendo],"[aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]","[recomendo, aparelho, eficiente, no, site, marca, do, aparelho, esta, impresso, como, desinfector, ao, chegar, esta, com, outro, nome, atualizar, com, marca, correta, uma, vez, que, mesmo, aparelho]",2018-05-22,2018-05-23 16:45:47
4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,1,[],"[mas, um, pouco, travando, pelo, valor, ta, boa]","[mas, um, pouco, travando, pelo, valor, ta, boa]",2018-02-16,2018-02-20 10:52:22
d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,1,[otimo],"[loja, nota]","[otimo, loja, nota]",2018-07-10,2018-07-11 14:10:25
0e0190b9db53b689b285d3f3916f8441,79832b7cb59ac6f887088ffd686e1d5e,5,1,[],"[obrigado, pela, atencao, amim, dispensada]","[obrigado, pela, atencao, amim, dispensada]",2017-12-01,2017-12-09 22:58:58


Divisão estratificada entre treinamento e teste

In [48]:
X = dlzd_olist_order_reviews.loc[:, ['review_comment_title_and_message', 'review_sentiment']]
y = dlzd_olist_order_reviews.loc[:, ['review_sentiment']]

In [49]:
X_train, X_test, _, _ = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=51
)

Salvamento em parquet do conjunto de treinamento

In [50]:
X_train.shape

(16512, 2)

In [51]:
X_train.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews_training.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=True,
)

Salvamento em parquet do conjunto de teste

In [52]:
X_test.shape

(4129, 2)

In [53]:
X_test.to_parquet(
    path='../dataset/delivery/dlzd_olist_order_reviews_test.parquet.snappy',
    engine='pyarrow',
    compression='snappy',
    index=True,
)